In [ ]:
import numpy as np
from src.data.api import *
import folium
from folium import plugins
import os
import pandas as pd
import json
import progressbar
import branca

html_save_path = '-'
dict_save_path = '-'

# Sensor IDs
sensor_ids = np.arange(99,10800)

In [ ]:
sensors = dict()
bar = progressbar.ProgressBar(maxval=len(sensor_ids)).start()
for idx, sensor_id in enumerate(sensor_ids):
    bar.update(idx)
    device = api_device(str(sensor_id), verbose = False)
    added_at = device.get_device_added_at()
    last_reading_at = device.get_date_last_reading()
    latlong = device.get_device_lat_long()
    coordinates = [latlong[1],latlong[0]]# for whatever reason, leaflet uses the coordinates in reverse
    kitID = device.get_kit_ID()
    sensors[str(sensor_id)]={'added_at':added_at, 'last_reading_at':last_reading_at, 'coordinates': coordinates, 'kitID': kitID}

In [ ]:
with open(dict_save_path, 'w') as outfile:
    json.dump(sensors, outfile)

In [ ]:
iSCAPE_IDs =[19, 20, 21, 28]
making_sense_IDs = [11, 14]
SCK_21_IDs = [26]

In [ ]:
with open(dict_save_path) as f:
    sensors = json.load(f)

In [ ]:
features = []
for sensor in sensors.keys():
    if sensors[sensor]['added_at'] is None: continue
    if sensors[sensor]['coordinates'] != [None, None] and sensors[sensor]['added_at']>'2001-01-01':
        if sensors[sensor]['last_reading_at'] is not None:
            datelist = pd.date_range(start=sensors[sensor]['added_at'], end=sensors[sensor]['last_reading_at'],normalize = True, freq='W').strftime('%Y-%m-%d')
            date_list = list()
            for item in datelist.values: date_list.append(str(item))
        else:
            date_list =[sensors[sensor]['added_at']]
        
        if sensors[sensor]['kitID'] in iSCAPE_IDs: color = '#7dbd4c'
        elif sensors[sensor]['kitID'] in making_sense_IDs: color = '#f88027'
        elif sensors[sensor]['kitID'] in SCK_21_IDs: color = '#ffb500'
        else: color = '#0019ff'

        features.append(
            {
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString',
                    'coordinates': [sensors[sensor]['coordinates']]*len(date_list),
                    'popup': popup,
                },
                'properties': {
                    'times': date_list,
                    'icon': 'circle',
                    'iconstyle': {
                        'fillColor': color,
                        'fillOpacity': 0.6,
                        'stroke': 'false',
                        'radius': 6
                    },
                    'style': {'weight': 0},
                    'id': 'man'
                }
            }
        )

In [ ]:
m = folium.Map(
    location=[41.3947688,2.0787279],
    tiles='Stamen Toner',
    zoom_start=2.5,
)

In [ ]:
m = folium.Map(
    location=[41.3947688,2.0787279],
    tiles='Stamen Toner',
    zoom_start=2.5,
)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1W',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=5,
    loop_button=True,
    date_options='YYYY/MM/DD',
    time_slider_drag_update=True,
    duration='P1W'
).add_to(m)

m.save(html_save_path)
# m